# QRPROG Final Project: Modular Exponentiation
## 18 January 2026
## Yahya Ali and Carmen Canedo

## General Notes
    - Looking to solve X**Y mod N
    - Central subroutine of **Shor's period finding algorithm** used for integer factorization

## Goal
Implement a quantum circuit for modular exponentiation from scratch.

# Limitations
    - Unless stated otherwise, you are only allowed to use the gates X, CNOT, CCNOT and Multicontrolled-NOT. See Appendix A for an example of use of the Multicontrolled-NOT gate.

    - Can implement additional auxiliary functions as long as the functions below are implemented.

    - For each implemented function, please give evidence that the implementation is correct by:
        - initializing each input register with some number up to 4 bits, 
        - each auxiliary register with |0>
        - measuring the output register to verify if the value is as expected

    - Reuse qubits from auxiliary registers as much as possible.
        - It is crucial that auxiliary registers are equal to |0> both at the beginning and at the end of computation of each function

# Libraries

In [6]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit import QuantumCircuit
from qiskit.circuit.library import MCXGate
from qiskit.providers.basic_provider import BasicSimulator
from qiskit_aer import AerSimulator
from qiskit import transpile

# 1.1 Initialization
The function `set_bits(circuit,A,X)` initializes the bits of register `A` with the binary string `X`.

For each i in `len(X)`, if `X[i]=1`, then the function applies the **X**-gate to `A[i]`.

Otherwise, it does nothing.

Assume `len(A)=len(X)`.

If `qubits = [2,4,3,7,5]` and `X = 01011`, the **X**-gate is applied to qubits 4, 7, and 5

In [7]:
def set_bits(circuit,A,X):

    # Width is determined by A
    w = len(A)
    
    # Check if X is read-in as a string or an integer
    # If X is written as int, X = 11
    if isinstance(X, int):

        # Check not negative
        if X < 0:
            raise ValueError("X must be a positive integer.")
        
        # Convert to binary
        X = bin(X)
        # Remove leading 0b
        X = X[2:]
        # Pad
        X = X.zfill(w)

    elif isinstance(X, str):
            
            # String handling
            if X.startswith("0b"):

                # X has already been wrapped by bin(), ie X = bin(11)
                # Strip 0b
                X = X[2:]
                # Pad
                X = X.zfill(w)
        
            else:

                # X is just the string "01011" or "1011"
                # and we are assuming that input for the decimal number "1,011" 
                # will jsut be written as X=1011, not its binary equivalent 1111110011
                X = X.zfill(w)

    else:

        raise TypeError("X must be an int or str")
    
    if len(X) > w:

        raise ValueError("Binary value doesn't fit in target register")

    # Enforce width
    X = X.zfill(w)

    # Reverse bit strong so A[0] is the least significant bit-index of the register
    X = X[::-1]

    # Apply X gate
    for i in range(w):
        
        if X[i]=="1":
            circuit.x(A[i])
        

    circuit.barrier()

    return circuit

## Check 1.1 `set_bits()`

In [8]:
# Circuit initialization
circuit = QuantumCircuit(8, 0)

# Qubits
A = [2,4,3,7,5]

# Options for X
# X = 01011, which is 11 in decimal
# X = bin(11) 
X = "1011"

# Call set_bits()
set_bits(circuit,A,X)

# Result
print(circuit)

           ░ 
q_0: ──────░─
           ░ 
q_1: ──────░─
     ┌───┐ ░ 
q_2: ┤ X ├─░─
     └───┘ ░ 
q_3: ──────░─
     ┌───┐ ░ 
q_4: ┤ X ├─░─
     └───┘ ░ 
q_5: ──────░─
           ░ 
q_6: ──────░─
     ┌───┐ ░ 
q_7: ┤ X ├─░─
     └───┘ ░ 


# 1.2 Copy
The function `copy(circuit,A,B)` copies the binary string `bin(A)` to register B.

Assume that `len(A)=len(B)` and before application of function, B is initialized to |0>

**Hint: use CNOT gates**

In [9]:
def copy(circuit,A,B):

    # Copy binary sting bin(A) to register B using CNOT gates
    for i in range(len(A)):
        circuit.cx(A[i], B[i])

    return circuit

## Check 1.2 `copy()`

In [11]:
# Initializations
circuit = QuantumCircuit(8, 4)
A = [0, 1, 2, 3]
B = [4, 5, 6, 7]

# Use set_bits() 
set_bits(circuit, A, 11)

# Some other possible inputs
# set_bits(circuit, A, "1011")
# set_bits(circuit, A, "01011")

# Copy A to B
copy(circuit,A,B)

# Resutls
print(circuit)

     ┌───┐ ░                     
q_0: ┤ X ├─░───■─────────────────
     ├───┤ ░   │                 
q_1: ┤ X ├─░───┼────■────────────
     └───┘ ░   │    │            
q_2: ──────░───┼────┼────■───────
     ┌───┐ ░   │    │    │       
q_3: ┤ X ├─░───┼────┼────┼────■──
     └───┘ ░ ┌─┴─┐  │    │    │  
q_4: ──────░─┤ X ├──┼────┼────┼──
           ░ └───┘┌─┴─┐  │    │  
q_5: ──────░──────┤ X ├──┼────┼──
           ░      └───┘┌─┴─┐  │  
q_6: ──────░───────────┤ X ├──┼──
           ░           └───┘┌─┴─┐
q_7: ──────░────────────────┤ X ├
           ░                └───┘
c: 4/════════════════════════════
                                 


# 1.3 Full Adder
The function `full_adder(circuit,a,b,r,c_in,c_out,AUX)` implements a full adder.

Registers:

`a` and `b` store the bits to the added

`c_in` stores the carry-in bit

`c_out` stores the carry-out bit

`r` stores the result of the sum

`AUX` is the auxiliary register

# 1.4 Addition
The function `add(circuit,A,B,R,AUX)` implements a circuit that adds `number(A)` to `number(B)` and stores the result at register `R`.

Assume `len(A)==len(B)==lent(R)`

The circuit is obtained by creating as cascade of `full_adder` circuits.

The carry bits are part of the auxiliary register AUX.

Note the carry-in bit of the first adder (from right to left) is set to 0.

# 1.5 Subtraction
The function `subtract(circuit,A,B,R,AUX)` implements a circuit that subtracts `Number(B)` from `Number(A)` and stores the result in the register `R`.

Assume that `len(A)=len(B)=len(R)`.

Such a circuit can be obtained by negating each bit stores in B, and applying the adder circuit with the first carry-in bit set to 1 instead of 0.

# 1.6 Comparison
The function `greater_or_eq(circuit,A,B,r,AUX)` implements a circuit that tests whether `number(A)` is greater than or equal to `number(B)`

Results is stored in register r.

Assume `len(A)=len(B)`

# 1.7 Addition Modulo N
The fucntion `add_mod(circuit,N,A,B,R,aux)` implements a circuit that adds `number(A)` to `number(B)` modulo `number(N)`.

Result is stored at register `R`.

Assume `len(A)==len(B)==len(N)`, `number(A) < number(N)`, and `number(B) < number(N)`

**HINT**
Since the numbers are stored in `A` and `B` are smaller than `N`, the sum will be SMALLER than 2*N.

It is enough to 1. add both numbers, 2. test whether the result is great_or_eq to N 3. If it greater than N, subtract N from result.

Test can be done by applying controlled subtraction (control bit is the  output bit of the comparison function)

## Helper functions
Keeping things concise. Want to make sure that all carried bits are calculated correctly and also reversible so that AUX always returns to 0, regardless of A and B vals.

### Not controlled

In [ ]:
# Bit-wise operations to be used in add_update

### Controlled
These only act if the control qubit = 1.

In [ ]:
# Initialize multicontrolled NOT gate

In [ ]:
# def add_mod()

In [ ]:
# def add_mod_inv()

# 1.8 Multiplication by Two Modulo N
The function `times_two_mod(circuit,N,A,R,AUX)` implements a circuit that doubles `number(A)` modulo `number(N)`

Result is stored in register `R`

**HINT** copy the register `A` and then compute `number(A) + number(A)`  modulo `number(N)`

# 1.9 Multiplication by a Power of Two Modulo N
The function `times_two_power_mod(circuit,N,A,k,R,AUX)` implements a circuit that multiplies `number(A)` by 2**k modulo

**HINT**: apply the function `times_two_mod(circuit,N,A,R,AUX)` k times in a row

# 1.10 Multiplication Modulo N
The function `multiply_mod(circuit,N,A,B,R,AUX)` implements a circuit that multiplies `number(A)` with `number(B)` modulo `number(N)`.

The result is stored in regiester `R`

**HINT** `number(A)` * `number(B)` == `number(A)` * sum of 2^k*`number(B)` from 0 to len(B) - 1

Apply a circuit that computes the partial sums for k varying between 0 and len(B) - 1

At each step `k`, apply the function `multiply_power_two_mod(circuit,N,A,k,R,AUX)` with control bit B[k] and. sum the result with the partial sum obtained at step k - 1

In [ ]:
# def mask_term()

# 1.11 Multiplcation modulo N with a hard-coded factor
The function `mutliply_mod_fixed(circuit,N,X,B,AUX)` implements a circuit that multiples `number(B)` by a fixed number X modulo N.

The multiplication should be done in place.

The circuit implements a unitary transformation that sends |number(B)⟩|0⟩to |X* number(B) mod N⟩|0⟩.

In [ ]:
# def set_const_bits()

# 1.12 Multiplication by X^(2^k) mod N
Let X be a fixed n-bit number. The function `multiply_mod_fixed_power_2_k(circuit,N,X,B,AUX,k)` implements a circuit that multiplies `number(B)` by the number `X^(2^k)` mod `N`

Instead of applying the function of the previous section 2k times, first pre-compute the
number `W = X^(2k)` mod N using python (not a quantum circuit)

Then call `multiply_mod_fixed(circuit, N, X, B, AUX)` that constructs the circuit that multiplies `number(B)` by W

Observe that the number W can be efficiently computed by taking the square of X mod N, then the square of the resulting number mod N and so on

In [ ]:
# Python-side helper

# 1.13 Multiplication by X^Y mod N
Let X be a fixed n-bit number. the function `multiply_mod_fixed_power_Y(circuit,N,X,B,AUX,Y)` implements a circuit that multiplies `number(B)` by the number `X^Y` modulo `N`, where `Y` is a given n-bit number. 

If Y= y_n−1...y_1y_0, then Y= sum of y_k·2^k from k=0 to n

For each k from 0 to n-1, biew the bit y_k as a control bit.

If this bit is 0, the circuit does nothing.

If it is 1, apply the function of section 1.12 to multiply the `number(B)` by `X^(2^k)` modulo N

In [ ]:
# def bits_set_positions()